In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline

In [30]:
class NNModel:
    
    def __init__(self,learning_rate, n_iter, args):
        self.learning_rate = learning_rate
        self.args = args
        self.n_iter = n_iter
    
    def z_score(self,w,x,b):
        return np.dot(w,x)+b
    
    def init_params(self,n_x):
        parameters={}
        n_a = n_x
        for i in range(1,len(self.args)+1):
            n_h = self.args[i-1][0]
            parameters['w'+str(i)] = np.random.rand(n_h,n_a)*np.sqrt(1/n_x)
            parameters['b'+str(i)] = np.random.randn(n_h,1)
            n_a = n_h
        return parameters
    
    def activation(self,z,fn = 'linear'):
        act_fn ={'linear':z,
                 'relu':np.maximum(z,0),
                 'tanh':np.tanh(z),
                 'sigmoid':1/(1+np.exp(-z)),
                 'softmax':np.exp(z)/np.sum(np.exp(z))}
        return act_fn[fn]
    
    def forward_prop(self,x, parameters):
        L = len(args)
        z_scores = {}
        activations = {'a0':x}
        for i in range(1,L+1):
            z_scores['z'+str(i)] = self.z_score(parameters['w'+str(i)],activations['a'+str(i-1)],parameters['b'+str(i)])
            z = z_scores['z'+str(i)]
            activations['a'+str(i)] = self.activation(z,fn=self.args[i-1][1])
        
        return z_scores, activations
    
    def compute_cost(self,y,y_hat):
        m = y.shape[0]
        cost = (-1/m)*(np.dot(y, np.log(y_hat.T+0.0000001)) + np.dot(1-y, np.log(1-y_hat.T+0.0000001)))
        return np.squeeze(cost)
        
    def backprop(self,y, parameters, z_scores, activations):
        gradients = {}
        L = len(self.args)
        m = y.shape[0]
        for i in range(L,0,-1):
            if i==L:
                gradients['dz'+str(i)]=activations['a'+str(i)]-y
            else:
                gradients['dz'+str(i)] = np.multiply(np.dot(parameters['w'+str(i+1)].T, gradients['dz'+str(i+1)]), 1*(z_scores['z'+str(i)]>=0))
            dz = gradients['dz'+str(i)]
            gradients['dw'+str(i)] = (1/m)*np.matmul(dz,activations['a'+str(i-1)].T)
            gradients['db'+str(i)] = (1/m)*np.sum(dz,axis=1,keepdims=True)
        return gradients
    
    def update_params(self,parameters, gradients):
        eta = self.learning_rate
        for i in range(1,len(parameters)//2+1):
            parameters['w'+str(i)]-=eta*gradients['dw'+str(i)]
            parameters['b'+str(i)]-=eta*gradients['db'+str(i)]
        return parameters
    
    def fit(self,x,y):
        np.random.seed(5)
        params = self.init_params(x.shape[0])
        for i in range(self.n_iter):
            z_scores,activations = self.forward_prop(x,params)
            y_hat = activations['a'+str(len(self.args))]
            #print(y_hat)
            cost = self.compute_cost(y,y_hat)
            gradients = self.backprop(y,params,z_scores,activations)
            params = self.update_params(params,gradients)
            if i%1000==0:
                print('Iteration : {}      Cost : {}'.format(i,cost))
        return params
    
    def predict(self,x_test,params):
        z_scores, activations = self.forward_prop(x_test,params)
        y_pred = 1*(activations['a'+str(len(params)//2)]>0.5)
        return np.squeeze(y_pred)

In [31]:
path = '/home/mrityunjay/Downloads/sonar.csv'
df = pd.read_csv(path)
df.columns=['x'+str(i) for i in range(len(df.columns))]
X=df.drop(['x60'],axis=1)
Y=df['x60']
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25,random_state=1)

X_train = np.transpose(X_train.values)
X_test = np.transpose(X_test.values)

Y_train = 1*(Y_train.values=='R')
Y_test = 1*(Y_test.values=='R')
Y_train = Y_train.reshape(1,Y_train.shape[0])
Y_test = Y_test.reshape(1,Y_test.shape[0])

In [41]:
args=[(100,'relu'),(50,'relu'),(10,'relu'),(5,'relu'),(3,'relu'),(1,'sigmoid')]
nn = NNModel(learning_rate=0.001, n_iter = 10000, args=args)
params = nn.fit(X_train,Y_train)

Iteration : 0      Cost : 110.57895885689418
Iteration : 1000      Cost : 97.20215085637494
Iteration : 2000      Cost : 24.107818263784335
Iteration : 3000      Cost : 0.059483096239265856
Iteration : 4000      Cost : 0.01442973769956715
Iteration : 5000      Cost : 0.007538193128651977
Iteration : 6000      Cost : 0.004934892738423426
Iteration : 7000      Cost : 0.00361163395157489
Iteration : 8000      Cost : 0.002820134439769308
Iteration : 9000      Cost : 0.0022975636960569958


In [42]:
Y_pred = nn.predict(X_test,params)
print(Y_pred)

[0 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1
 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0]


In [43]:
print(Y_test)

[[0 0 0 0 1 1 0 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 0 0 0 1 1 1 1
  1 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0]]


In [44]:
acc = accuracy_score(Y_pred,np.squeeze(Y_test))
print(acc)

0.8461538461538461
